In [141]:
import random
import math
import pandas as pd
import numpy as np

In [142]:
class City:
    def __init__(self, xCord, yCord, zCord):
        self.xCord = xCord
        self.yCord = yCord
        self.zCord = zCord
        
    def displayCityCoordinates(self):
        print('('+str(self.xCord)+','+str(self.yCord)+','+str(self.zCord)+')')
        
        
def distanceBetweenTepCities(fromCity, toCity):
    return(math.sqrt(abs(fromCity.xCord - toCity.xCord)**2 + abs(fromCity.yCord - toCity.yCord)**2 + abs(fromCity.zCord - toCity.zCord)**2))

In [143]:
class route:
    def __init__(self, route):
        self.route = route
        self.route_distance = 0
        self.route_fitness = 0.0
        
    def findDistance(self):
        if self.route_distance == 0:
            distance = 0
            for i in range(len(self.route)):
                fromCity = self.route[i]
                if i+1 < len(self.route):
                    toCity = self.route[i+1]
                else:
                    toCity = self.route[0]
                distance += distanceBetweenTepCities(fromCity, toCity)
            self.route_distance = distance
                
        
    def fitness(self):
        if self.route_fitness == 0.0:
            self.findDistance()
            self.route_fitness = 1/(self.route_distance)
        return self.route_fitness

In [163]:
def readInputFile(fileName):
    noOfCities = None
    inputList = []
    countFlag = 0
    cities = []

    with open(fileName) as f:
        for line in f:
            if countFlag == 0:
                countFlag = 1
                noOfCities = int(line[:-1])
            else:
                coordinates = line[:-1].split(' ')
                city = City(int(coordinates[0]), int(coordinates[1]), int(coordinates[2]))
                cities.append(city)

    return noOfCities, cities

In [145]:
def createRandomRoutes(citiesList):
    citiesList = random.sample(citiesList, len(citiesList))
    return citiesList

In [146]:
def createInitialPopulation(citiesList, populationSize):
    population = [citiesList]
    
    for i in range(populationSize-1):
        population.append(createRandomRoutes(citiesList))
    
    return population

In [147]:
def findFitness(population):
    populationFitness = {}
    
    for i in range(len(population)):
        populationFitness[i] = route(population[i]).fitness()
        
    return sorted(populationFitness.items(), key = lambda item:item[1], reverse = True)

In [148]:
def parentSelection(population):
    parents = []
    
    populationdf = pd.DataFrame(np.array(population), columns=["Index","Fitness"])
    populationdf['cum_sum'] = populationdf.Fitness.cumsum()
    
    totalSum = populationdf['Fitness'].sum()
    
    for i in range(int(len(population)*0.2)):
        parents.append(population[i])
        
    for i in range(len(population) - int(len(population)*0.2)):
        randomValue = random.uniform(0,totalSum)
        
        df = populationdf[populationdf['cum_sum'] > randomValue]
        index = df.iat[0,0]
        populationIndex = populationdf[ populationdf['Index'] == index ].index[0]
        
        parents.append(population[populationIndex])
        
    return parents

In [149]:
def crossOver(parent1, parent2):
    startIndex = int(random.uniform(0,len(parent1)))
    endIndex = int(random.uniform(0,len(parent1)))
    
    if startIndex == endIndex:
        if startIndex == (len(parent1) - 1):
            endIndex = startIndex - 1
        else:
            endIndex = startIndex + 1
    
    sIndex = min(startIndex,endIndex)
    eIndex = max(endIndex,startIndex)
    
    child1 = parent1[sIndex:eIndex]
    
    child2 = []
    if sIndex != 0:
        for city in parent2:
            if city not in child1:
                if len(child2) != sIndex:
                    child2.append(city)
                else:
                    break
    child = child1 + child2
    
    child3 = [city for city in parent2 if city not in child]
    child += child3
    
    return child
    

In [150]:
def breedPopulation(populationPool):
    children = []
    
    for i in range(int(len(populationPool)*0.2)):
        children.append(populationPool[i])
        
    shufflePool = random.sample(populationPool, len(populationPool))
    
    for i in range(len(populationPool) - int(len(populationPool)*0.2)):
        ind = int(random.uniform(0,len(shufflePool)))
        if ind == i:
            if i == 0:
                ind += 1
            else:
                ind -= 1
        child = crossOver(shufflePool[i], shufflePool[ind])
        children.append(child)
        
    return children
        
    

In [151]:
def createParentPool(parents, population):
    parentPool = []
    
    for i in range(len(parents)):
        ind = parents[i][0]
        parentPool.append(population[ind])
        
    return parentPool
    

In [164]:
def displayRoute(route):
    for i in route:
        print('('+str(i.xCord)+','+str(i.yCord)+','+str(i.zCord)+')')

In [176]:
noOfCities, citiesList = readInputFile('input.txt')

currentGeneration = createInitialPopulation(citiesList,100)

for i in range(100):
    currentGenerationFitness = findFitness(currentGeneration)
    parents = parentSelection(currentGenerationFitness)
    parentPool = createParentPool(parents, initialPopulation)
    nextGeneration = breedPopulation(parentPool)
    currentGeneration = nextGeneration
    

print('distance:'+str(1/findFitness(currentGeneration)[0][1]))
displayRoute(currentGeneration[findFitness(currentGeneration)[0][0]])

distance:180.09086218218096
(60,103,97)
(61,103,97)
(62,103,97)
(63,103,97)
(64,103,97)
(60,103,9)
